In [15]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import os
import dotenv

dotenv.load_dotenv()

base_path = os.getenv("BASE_PATH", ".")
API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
print(base_path)
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    openai_api_key=API_KEY,
)

vectordb_path = r"C:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0606\chroma_db"


vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectordb_path,
    embedding_function=embedding,
)

vectorstore.delete_collection()

vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectordb_path,
    embedding_function=embedding,
)


C:\Users\Sese\autosave\알렌 이론 추출\theory_texts


In [16]:
from tqdm import tqdm
import os
for path in tqdm(os.listdir(base_path)):
   html_loader = UnstructuredHTMLLoader(
      file_path=os.path.join(base_path, path),
      mode="single",
   )
   documents = html_loader.load()
   for doc in documents:
      doc.metadata['theoryId'] = path.split('_')[1]
      doc.metadata['subChapterId'] = path.split('_')[0]
   vectorstore.add_documents(documents)

 96%|█████████▌| 980/1026 [09:15<00:26,  1.76it/s]


TypeError: Invalid input object: NoneType

In [ ]:
all_files = os.listdir(base_path)
files_to_process = all_files[980:]  # 에러 났던 파일부터 처리

for path in tqdm(files_to_process):
    try:
        html_loader = UnstructuredHTMLLoader(
            file_path=os.path.join(base_path, path),
            mode="single",
        )
        documents = html_loader.load()
        for doc in documents:
            doc.metadata['theoryId'] = path.split('_')[1]
            doc.metadata['subChapterId'] = path.split('_')[0]
        vectorstore.add_documents(documents)
    except Exception as e:
        print(f"[ERROR] 처리 실패: {path}")
        import traceback
        traceback.print_exc()


In [47]:
print(len(vectorstore.get()['documents']))
data = vectorstore.get()
for i, (doc, meta) in enumerate(zip(data["documents"], data["metadatas"])):
    print(f"[{i+1}] {doc[:100]}...")  # 문서 앞부분만 출력
    print(f"    metadata: {meta['source'].split('\\')[-1]}")


1023
[1] 감염성 질환에서 주로 동반되는 증상인 발열과 불명열에 대해 다룬다. 시험에 특별히 많이 출제되는 부분은 아니다. 불명열 환자에서 다음에 해야할 검사, 조치를 묻는 문제가 주로 출제...
    metadata: 1636_3826_발열, 불명열.html
[2] 그람양성, 그람음성 세균 분류와 항생제에 대해 다룬다. 이 파트에서 문제가 직접 출제되는 경우는 거의 없지만 감염내과를 공부하는데 가장 기본이 되는 내용이므로 잘 알아두자. 항생제...
    metadata: 1636_3827_항생제.html
[3] : Sepsis

국시에서 감염질환의 비중은 매년 15% 가까이 되며, 이 중 중증 감염도 많이 출제된다. 따라서 패혈증을 잘 이해하는 것이 국시에 있어서, 더 나아가 좋은 의사가...
    metadata: 1636_4736_패혈증.html
[4] : Osteomyelitis, 뼈속질염

지역사회 감염증(community-acquired infection) 단원에서는 여러 종류의 병원체가 공통적으로 일으킬 수 있는 질환들을 ...
    metadata: 1637_3860_골수염.html
[5] 지역사회 감염증에서 가장 흔한 설사와 식중독에 대해 다룬다. 일반적인 특징에 대해 전반적으로 살펴보고 각론으로 살모넬라, 이질, 캠필로박터, 콜레라에 대해서 자세히 다루겠다. 설사...
    metadata: 1637_3861_설사, 식중독.html
[6] : 위막성 대장염, Pseudomembranous colitis, PMC, Clostridium difficile infection, CDI

거의 매년 출제된다. 항생제 사용 과...
    metadata: 1638_3851_거짓막 결장염.html
[7] 병원감염(원내감염)과 의료관련감염에 대해 다룬다. 시험에 자주 출제되는 부분은 아니지만 임상에서 매우 중요하므로 의료관련감염의 중요성, 종류, 치료 및 예방법에 대해 잘 정리해두자...
    metadata: 1638_3852_Healt

In [48]:
import yaml
import os

# 벡터스토어와 retriever 세팅 (당신 코드와 동일)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

print(os.getcwd())
# YAML 파일에서 쿼리 로딩
with open("queries.yaml", "r", encoding="utf-8") as f:
    queries = yaml.safe_load(f).get("queries", [])

c:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0606


In [49]:
query_log = []

for query in queries:
    docs = retriever.invoke(query)
    print(f"\n🔍 Query: {query}")

    result_entry = {"query": query, "results": []}
    
    for doc in docs:
        source = doc.metadata.get("source", "Unknown")
        excerpt = doc.page_content[:200].replace("\n", " ").strip()
        result_entry["results"].append({
            "source": os.path.basename(source),
            "excerpt": excerpt
        })
        print(f"  - {source}")

    query_log.append(result_entry)

# 결과 저장
with open("query_log.yaml", "w", encoding="utf-8") as f:
    yaml.dump(query_log, f, allow_unicode=True)
    
    import yaml

with open("query_log.yaml", "r", encoding="utf-8") as f:
    log_data = yaml.safe_load(f)

lines = ["# 🔍 RAG Query Log\n"]

for entry in log_data:
    query = entry.get("query", "")
    results = entry.get("results", [])

    lines.append(f"\n## 🧪 Query: **{query}**\n")
    lines.append("| Rank | Source | Excerpt |")
    lines.append("|------|--------|---------|")

    for i, result in enumerate(results, 1):
        source = result.get("source", "N/A")
        excerpt = result.get("excerpt", "").replace("|", "\\|")  # Markdown-safe
        lines.append(f"| {i} | `{source}` | {excerpt} |")

# Markdown 파일 저장
with open("query_log.md", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print("✅ query_log.md 생성 완료!")



🔍 Query: 뎅기열과 비슷한 매개체로 인한 감염병은?
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1643_3848_여행 관련 바이러스 감염.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2522_5676_Leptospira interrogans.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2521_5659_Legionella pneumophila.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1637_3860_골수염.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2747_5639_염증의 양상.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2525_5685_Rubella virus.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1636_3826_발열, 불명열.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1638_3852_Healthcare-associated infections.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2525_5689_Norovirus.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1639_3828_피부연조직 감염.html

🔍 Query: 불명열 진단 순서를 설명하시오
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1636_3826_발열, 불명열.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2417_5489_근육 수축의 기계적 특성.html
  - C